# Material3 Migration for Flutter Panels

This notebook will guide through the process of migrating CollectionPropertyPanel and ImagePropertyPanel to Material3 with localization support.

## Setup Environment

First, let's ensure we have the necessary dependencies and setup for the migration.

In [ ]:
# Check Flutter version and packages
flutter --version
flutter pub get

We need to update the `pubspec.yaml` to ensure we have the required dependencies for Material3 and localization.

In [ ]:
# Add to pubspec.yaml
dependencies:
  flutter:
    sdk: flutter
  flutter_localizations:
    sdk: flutter
  intl: ^0.18.0
  # Other dependencies for your app
  
flutter:
  generate: true # Enable generation of localization files

## Migrate CollectionPropertyPanel

Let's start by migrating the CollectionPropertyPanel to Material3. We'll create a new file with the prefix 'm3_' and update the UI elements to follow Material3 design guidelines.

In [ ]:
// File: lib/widgets/m3_collection_property_panel.dart

import 'package:flutter/material.dart';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';

class M3CollectionPropertyPanel extends StatefulWidget {
  final List<dynamic> items;
  final Function(List<dynamic>) onItemsChanged;
  final String title;

  const M3CollectionPropertyPanel({
    Key? key,
    required this.items,
    required this.onItemsChanged,
    required this.title,
  }) : super(key: key);

  @override
  State<M3CollectionPropertyPanel> createState() => _M3CollectionPropertyPanelState();
}

class _M3CollectionPropertyPanelState extends State<M3CollectionPropertyPanel> {
  @override
  Widget build(BuildContext context) {
    final l10n = AppLocalizations.of(context)!;
    
    return Card(
      elevation: 0, // Material3 uses less elevation
      shape: RoundedRectangleBorder(
        borderRadius: BorderRadius.circular(12), // Material3 rounded corners
        side: BorderSide(color: Theme.of(context).colorScheme.outline), // Material3 outlined card
      ),
      child: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              widget.title,
              style: Theme.of(context).textTheme.titleMedium,
            ),
            const SizedBox(height: 16),
            ListView.builder(
              shrinkWrap: true,
              physics: const NeverScrollableScrollPhysics(),
              itemCount: widget.items.length,
              itemBuilder: (context, index) {
                return Padding(
                  padding: const EdgeInsets.only(bottom: 8.0),
                  child: Row(
                    children: [
                      Expanded(
                        child: Text(widget.items[index].toString()),
                      ),
                      IconButton(
                        icon: const Icon(Icons.delete_outline),
                        tooltip: l10n.removeItem,
                        onPressed: () {
                          setState(() {
                            final newItems = List<dynamic>.from(widget.items);
                            newItems.removeAt(index);
                            widget.onItemsChanged(newItems);
                          });
                        },
                      ),
                    ],
                  ),
                );
              },
            ),
            const SizedBox(height: 8),
            FilledButton.icon(
              icon: const Icon(Icons.add),
              label: Text(l10n.addItem),
              onPressed: () {
                // Logic to add a new item
                showDialog(
                  context: context,
                  builder: (context) {
                    String newItem = '';
                    return AlertDialog(
                      title: Text(l10n.addNewItem),
                      content: TextField(
                        onChanged: (value) => newItem = value,
                        decoration: InputDecoration(
                          hintText: l10n.enterItemName,
                        ),
                      ),
                      actions: [
                        TextButton(
                          onPressed: () => Navigator.of(context).pop(),
                          child: Text(l10n.cancel),
                        ),
                        FilledButton(
                          onPressed: () {
                            if (newItem.isNotEmpty) {
                              setState(() {
                                final newItems = List<dynamic>.from(widget.items);
                                newItems.add(newItem);
                                widget.onItemsChanged(newItems);
                              });
                              Navigator.of(context).pop();
                            }
                          },
                          child: Text(l10n.add),
                        ),
                      ],
                    );
                  },
                );
              },
            ),
          ],
        ),
      ),
    );
  }
}

## Migrate ImagePropertyPanel

Now, let's migrate the ImagePropertyPanel to Material3 with support for localization.

In [ ]:
// File: lib/widgets/m3_image_property_panel.dart

import 'package:flutter/material.dart';
import 'dart:io';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';
import 'package:image_picker/image_picker.dart';

class M3ImagePropertyPanel extends StatefulWidget {
  final String? imagePath;
  final Function(String?) onImageChanged;
  final String title;

  const M3ImagePropertyPanel({
    Key? key,
    this.imagePath,
    required this.onImageChanged,
    required this.title,
  }) : super(key: key);

  @override
  State<M3ImagePropertyPanel> createState() => _M3ImagePropertyPanelState();
}

class _M3ImagePropertyPanelState extends State<M3ImagePropertyPanel> {
  final ImagePicker _picker = ImagePicker();

  Future<void> _pickImage() async {
    final XFile? image = await _picker.pickImage(source: ImageSource.gallery);
    if (image != null) {
      widget.onImageChanged(image.path);
    }
  }

  @override
  Widget build(BuildContext context) {
    final l10n = AppLocalizations.of(context)!;
    
    return Card(
      elevation: 0, // Material3 uses less elevation
      shape: RoundedRectangleBorder(
        borderRadius: BorderRadius.circular(12), // Material3 rounded corners
        side: BorderSide(color: Theme.of(context).colorScheme.outline), // Material3 outlined card
      ),
      child: Padding(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              widget.title,
              style: Theme.of(context).textTheme.titleMedium,
            ),
            const SizedBox(height: 16),
            if (widget.imagePath != null)
              Stack(
                alignment: Alignment.topRight,
                children: [
                  ClipRRect(
                    borderRadius: BorderRadius.circular(8),
                    child: Image.file(
                      File(widget.imagePath!),
                      width: double.infinity,
                      height: 200,
                      fit: BoxFit.cover,
                    ),
                  ),
                  Padding(
                    padding: const EdgeInsets.all(8.0),
                    child: CircleAvatar(
                      backgroundColor: Theme.of(context).colorScheme.primary,
                      child: IconButton(
                        icon: Icon(
                          Icons.delete_outline,
                          color: Theme.of(context).colorScheme.onPrimary,
                        ),
                        tooltip: l10n.removeImage,
                        onPressed: () {
                          widget.onImageChanged(null);
                        },
                      ),
                    ),
                  ),
                ],
              )
            else
              Container(
                width: double.infinity,
                height: 200,
                decoration: BoxDecoration(
                  color: Theme.of(context).colorScheme.surfaceVariant,
                  borderRadius: BorderRadius.circular(8),
                  border: Border.all(color: Theme.of(context).colorScheme.outline),
                ),
                child: Center(
                  child: Text(l10n.noImageSelected),
                ),
              ),
            const SizedBox(height: 16),
            SizedBox(
              width: double.infinity,
              child: FilledButton.icon(
                icon: const Icon(Icons.photo_library),
                label: Text(widget.imagePath == null ? l10n.selectImage : l10n.changeImage),
                onPressed: _pickImage,
              ),
            ),
          ],
        ),
      ),
    );
  }
}

## Generate l10n Entries

Now we need to set up localization for our app. First, let's create the localization configuration file.

In [ ]:
// File: lib/l10n/l10n.dart

import 'package:flutter/material.dart';

class L10n {
  static final all = [
    const Locale('en'), // English
    const Locale('es'), // Spanish
    const Locale('fr'), // French
    // Add more locales as needed
  ];
}

Now, let's create the base localization file with all the strings we need for our migrated panels.

In [ ]:
// File: lib/l10n/app_en.arb

{
  "@@locale": "en",
  
  "removeItem": "Remove item",
  "@removeItem": {
    "description": "Tooltip for the button to remove an item from the collection"
  },
  
  "addItem": "Add item",
  "@addItem": {
    "description": "Label for the button to add an item to the collection"
  },
  
  "addNewItem": "Add new item",
  "@addNewItem": {
    "description": "Title for the dialog to add a new item"
  },
  
  "enterItemName": "Enter item name",
  "@enterItemName": {
    "description": "Hint text for the input field for a new item name"
  },
  
  "cancel": "Cancel",
  "@cancel": {
    "description": "Label for the cancel button"
  },
  
  "add": "Add",
  "@add": {
    "description": "Label for the add button in dialogs"
  },
  
  "selectImage": "Select image",
  "@selectImage": {
    "description": "Label for the button to select an image"
  },
  
  "changeImage": "Change image",
  "@changeImage": {
    "description": "Label for the button to change an existing image"
  },
  
  "removeImage": "Remove image",
  "@removeImage": {
    "description": "Tooltip for the button to remove an image"
  },
  
  "noImageSelected": "No image selected",
  "@noImageSelected": {
    "description": "Text shown when no image is selected"
  }
}

Let's add a Spanish translation file as an example:

In [ ]:
// File: lib/l10n/app_es.arb

{
  "@@locale": "es",
  
  "removeItem": "Eliminar elemento",
  "addItem": "Añadir elemento",
  "addNewItem": "Añadir nuevo elemento",
  "enterItemName": "Introduce el nombre del elemento",
  "cancel": "Cancelar",
  "add": "Añadir",
  "selectImage": "Seleccionar imagen",
  "changeImage": "Cambiar imagen",
  "removeImage": "Eliminar imagen",
  "noImageSelected": "Ninguna imagen seleccionada"
}

Now, let's update the main.dart file to integrate localization:

In [ ]:
// Update main.dart

import 'package:flutter/material.dart';
import 'package:flutter_localizations/flutter_localizations.dart';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';
import 'l10n/l10n.dart';

void main() {
  runApp(const MyApp());
}

class MyApp extends StatelessWidget {
  const MyApp({Key? key}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'M3 Migration Demo',
      theme: ThemeData(
        useMaterial3: true, // Enable Material3
        colorScheme: ColorScheme.fromSeed(seedColor: Colors.blue),
      ),
      supportedLocales: L10n.all,
      localizationsDelegates: const [
        AppLocalizations.delegate,
        GlobalMaterialLocalizations.delegate,
        GlobalWidgetsLocalizations.delegate,
        GlobalCupertinoLocalizations.delegate,
      ],
      home: const HomePage(),
    );
  }
}

class HomePage extends StatelessWidget {
  const HomePage({Key? key}) : super(key: key);

  @override
  Widget build(BuildContext context) {
    // Implementation of home page
    return Scaffold(
      // ...
    );
  }
}

## Verify Localization Integration

Let's create a test page to verify that our migrated panels work correctly with localization:

In [ ]:
// File: lib/test_page.dart

import 'package:flutter/material.dart';
import 'package:flutter_gen/gen_l10n/app_localizations.dart';
import 'widgets/m3_collection_property_panel.dart';
import 'widgets/m3_image_property_panel.dart';

class TestPage extends StatefulWidget {
  const TestPage({Key? key}) : super(key: key);

  @override
  State<TestPage> createState() => _TestPageState();
}

class _TestPageState extends State<TestPage> {
  List<String> _items = ['Item 1', 'Item 2', 'Item 3'];
  String? _imagePath;

  @override
  Widget build(BuildContext context) {
    final l10n = AppLocalizations.of(context)!;
    
    return Scaffold(
      appBar: AppBar(
        title: Text('Material3 Panels Test'),
      ),
      body: SingleChildScrollView(
        padding: const EdgeInsets.all(16.0),
        child: Column(
          crossAxisAlignment: CrossAxisAlignment.start,
          children: [
            Text(
              'Testing Material3 Panels with Localization',
              style: Theme.of(context).textTheme.headlineSmall,
            ),
            const SizedBox(height: 24),
            
            // Test CollectionPropertyPanel
            M3CollectionPropertyPanel(
              items: _items,
              onItemsChanged: (newItems) {
                setState(() {
                  _items = List<String>.from(newItems);
                });
              },
              title: 'Collection Panel',
            ),
            
            const SizedBox(height: 24),
            
            // Test ImagePropertyPanel
            M3ImagePropertyPanel(
              imagePath: _imagePath,
              onImageChanged: (newPath) {
                setState(() {
                  _imagePath = newPath;
                });
              },
              title: 'Image Panel',
            ),
            
            const SizedBox(height: 32),
            
            // Language Switcher
            Text(
              'Switch Language:',
              style: Theme.of(context).textTheme.titleMedium,
            ),
            const SizedBox(height: 8),
            Wrap(
              spacing: 8,
              children: [
                ElevatedButton(
                  onPressed: () {
                    // Change locale to English
                    Locale newLocale = const Locale('en');
                    Localizations.override(
                      context: context,
                      locale: newLocale,
                      child: const MyApp(),
                    );
                  },
                  child: const Text('English'),
                ),
                ElevatedButton(
                  onPressed: () {
                    // Change locale to Spanish
                    Locale newLocale = const Locale('es');
                    Localizations.override(
                      context: context,
                      locale: newLocale,
                      child: const MyApp(),
                    );
                  },
                  child: const Text('Español'),
                ),
              ],
            ),
          ],
        ),
      ),
    );
  }
}

To verify that everything is working correctly, you should:

1. Run `flutter pub get` to ensure all dependencies are installed
2. Run `flutter gen-l10n` to generate the localization files
3. Launch the app and navigate to the test page
4. Test both panels with different languages to verify localization
5. Verify that the UI follows Material3 design guidelines

The migration is now complete! You've successfully migrated the CollectionPropertyPanel and ImagePropertyPanel to Material3 with proper localization support.